In [1]:
#Standard libraries
import os
import time
import sys
#Third party libraries
import rasterio
import rasterio.plot
import rasterio.mask
import rasterio.io
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(legacy='1.25')
import geopandas
import pandas
import fiona
from tqdm.notebook import tqdm
#Local applications
package_dir = os.path.dirname(os.getcwd())
if package_dir not in sys.path:
    sys.path.insert(0, package_dir)
from hpr_detection_toolkit import utils
from hpr_detection_toolkit.line_detection import LineSegmentDetector
from hpr_detection_toolkit.hpr_detection import HprDitchDetector

# Setting up
First we define the coordinate reference system in which we want to work in. 

In [2]:
target_crs = "EPSG:31370"

Next we get we load the necessary data.

In this example, we will use the vector data of the biological value map (BWK) and the agricultural usages plots (Lgp or Lbgebrperc), and the raster data of the VITO AI map regarding microrelief. We load this data using the `utils` module of this package, which will check if the crs of data set and performs a reprojection on the fly if needed.

Note: to speed up the calculations, we limit this example to KB20 of the VITO raster map. Therefore, we preselected the agricultural usage plots located only within the boundaries of this raster map using QGIS. We saved both the raster map, the polygon representing the rasters boundary and the landplots in the `KB20.gpkg` geopackage file.

> **Adjust the filepath for your own local set-up!**

In [3]:
lgp = utils.open_vector_data('data/KB20.gpkg', layer='Lbgebrperc2016_2023', target_crs=target_crs)

Vector data in data/KB20.gpkg in EPSG:31370, no reprojection.


In [4]:
VITO_map = utils.open_raster_data('data/KB20.gpkg', layer='VITO_predicted_logits', target_crs=target_crs)
#VITO_map = utils.open_raster_data('E:/Stage INBO/Data/VITO_microrelief/KB20_predicted_logits.tif', target_crs=target_crs)

Raster data in GPKG:data/KB20.gpkg:VITO_predicted_logits in EPSG:31370, no reprojection.


# Selecting the grasslands
### Masking the search region in the BWK
Next, we will filter the data and select the right grasslands to perform the analyses on. 

1. We should only analyse grasslands that are not a habitat (HAB1 contains `gh` or `rbb`)
2. Grasslands that are already categorised as `hpr`of `hpr+` in EENH1, don't need to be analysed

To easily work with the search and nosearch region, we unify all the polygons into one geometry and store it as a geopandas.DataFrame. This step may already been preprocessed in the past. Therefore we check if the file already exist, and if not we calculate it and store it for future analyses.

In [5]:
preprocessing_filename = 'data/DetectionMicrorelief_preprocessing.gpkg'
preprocessed_layers = {}

layers_to_load = ['search_region', 'nosearch_region']
if os.path.isfile(preprocessing_filename):
    print(f'Preprocessed file detected.')
    print(f'Checking layers in {preprocessing_filename}')
    for layer_name in layers_to_load:
        if layer_name in fiona.listlayers(preprocessing_filename):
            preprocessed_layers[layer_name] = geopandas.read_file(preprocessing_filename, layer=layer_name)
            print(f'Layer {layer_name} found in {preprocessing_filename} and loaded in')

for layer_name in layers_to_load:
    if layer_name not in preprocessed_layers.keys():
        print(f'Layer {layer_name} not found in {preprocessing_filename}')
        if layer_name == 'nosearch_region':
            nosearch_region = bwk[~mask].geometry.union_all()
            preprocessed_layer['nosearch_region'] = geopandas.GeoDataFrame(geometry=[nosearch_region], crs=target_crs)
            preprocessed_layer['nosearch_region'].to_file(preprocessing_filename, driver='GPKG', mode='a', layer='nosearch_region')
            print(f'Layer {layer_name} calculated and saved to {preprocessing_filename}')
        if layer_name == 'search_region':
            search_region = bwk[mask].geometry.union_all()
            preprocessed_layer['search_region'] = geopandas.GeoDataFrame(geometry=[search_region], crs=target_crs)
            preprocessed_layer['search_region'].to_file(preprocessing_filename, driver='GPKG', mode='a', layer='search_region')
            print(f'Layer {layer_name} calculated and saved to {preprocessing_filename}')

Preprocessed file detected.
Checking layers in data/DetectionMicrorelief_preprocessing.gpkg
Layer search_region found in data/DetectionMicrorelief_preprocessing.gpkg and loaded in
Layer nosearch_region found in data/DetectionMicrorelief_preprocessing.gpkg and loaded in


### Selecting the grasslands to analyse
The plots that are grassland can be obtained from the agriculture usage. We look only at permanent grassland, meaning the landplot has always been categorised as grassland during a certain time period (here 2016 to 2022)

In [6]:
mask_grasslands = lgp['lgp_7j_BWK'] == 'Permanent grasland - hp'
grasslands = lgp[mask_grasslands]
print(f'KB20 contains {len(lgp)} landplots, of which {len(grasslands)} ({len(grasslands)/len(lgp)*100:.0f}%) are permanent grasslands')

KB20 contains 44999 landplots, of which 10882 (24%) are permanent grasslands


In [7]:
grasslands_to_inspect = geopandas.read_file('DetectionMicrorelief_KB20.gpkg', layer='grassland_selection')
print(f'KB20 contains {len(grasslands)} permanent grasslands, of which {len(grasslands_to_inspect)}' + 
      f' ({len(grasslands_to_inspect)/len(grasslands)*100:.0f}%) need to be analysed.')

KB20 contains 10882 permanent grasslands, of which 7816 (72%) need to be analysed.


# Analysing the selected grasslands
The categorisation of the grassland is based on the presence of ditches in the landplot. When a buffer zone around the ditches of 30 m covers 70% of the landplot's area, the grassland can be categorised as HPR. Therefore, we will try to detect the ditches and calculate the buffer fraction, which we store as a new attribute for each landplot. 

First we add this new attribute to the table for each selected grasslands and set its value to nan.

In [8]:
grasslands_to_inspect['ditch_buffer_fraction'] = np.nan

To performs the necessary processing step to calculate areal fraction of the buffer zone, the `HprDitchDetector` class from this package can be used

To do so, the `HprDitchDetector` uses some default configuration, which are stored in a yaml file. Nevertheless, these default configurations can be overwritten during initialisation of a new object by including a user-defined configuration (dictionary format).

To detect the ditches, the `HprDitchDetector` relies on a `LineSegmentDetector` object. This also uses default configuration saved in the samen yaml file, which can be overwritten similarly.

In [9]:
lsd = LineSegmentDetector()
user_config = {'buffer_zone': {
                    'distance': 15.},
               'filter_background': {
                    'threshold_value': .5*255}
              }
hpr_detector = HprDitchDetector(VITO_map, lsd, config=user_config)

To calculate the buffer_fraction, we iterate through all the selected landplots and perform the processing. For debugging purposes we will also keep the geometry of the detected ditches and buffer zones.

In [ ]:
ditches = [None] *len(grasslands_to_inspect) #geopandas.GeoDataFrame(columns=['geometry'], crs=target_crs)
buffer_zones = [None] *len(grasslands_to_inspect) #geopandas.GeoDataFrame(columns=['geometry'], crs=target_crs)

for i in tqdm(range(len(grasslands_to_inspect)), desc="Processeing selected grasslands"):
    index = grasslands_to_inspect.index[i]
    landplot = grasslands_to_inspect.iloc[i:i+1]
    hpr_detector.process(landplot)
    #ditches = pandas.concat([ditches, hpr_detector.get_ditches(multilinestring=True)], ignore_index=True)
    #buffer_zones = pandas.concat([buffer_zones, hpr_detector.get_buffer_zone()], ignore_index=True)
    ditches[i] = hpr_detector.get_ditches(multilinestring=True).geometry
    buffer_zones[i] = hpr_detector.get_buffer_zone().geometry
    grasslands_to_inspect.loc[index,'ditch_buffer_fraction'] = hpr_detector.get_hpr_fraction()

ditches = [ditch.loc[0] for ditch in ditches]
ditches = geopandas.GeoDataFrame(geometry=ditches, crs=target_crs)
buffer_zones = [buffer_zone.loc[0] if len(buffer_zone) != 0 else None for buffer_zone in buffer_zones]
buffer_zones = geopandas.GeoDataFrame(geometry=buffer_zones, crs=target_crs)

print(f'Of the {len(grasslands_to_inspect)} inspected grasslands in KB20,' + 
      f'{len(grasslands_to_inspect[grasslands_to_inspect.ditch_buffer_fraction > .65])}' + 
      f'of them are hpr candidates based on ditch detection during this processing')

Processeing selected grasslands:   0%|          | 0/7816 [00:00<?, ?it/s]

Once the processing is finished, we save the inspected grasslands with the computed buffer fraction in a new layer and also the line segments representing the ditches and the buffer zone within the landplot.

In [ ]:
grasslands_to_inspect.to_file('DetectionMicrorelief_KB20.gpkg', driver='GPKG', layer='grasslands_processed')
ditches.to_file('DetectionMicrorelief_KB20.gpkg', driver='GPKG', layer='ditches')
buffer_zones.to_file('DetectionMicrorelief_KB20.gpkg', driver='GPKG', layer='buffer_zones')

In [ ]:
# to read in previous processed landplots
# grasslands_to_inspect = utils.open_vector_data('DetectionMicrorelief_KB20.gpkg', layer='grasslands_processed')
# ditches = utils.open_vector_data('DetectionMicrorelief_KB20.gpkg', layer='ditches')
# buffer_zones = utils.open_vector_data('DetectionMicrorelief_KB20.gpkg', layer='buffer_zones')

Lastly, to visually inspect the landplots easily, you can make a combined plot of the VITO_map and the detected ditches for landplots with a certain buffer fraction. To do this, make the cell below a code cell (Run > Cell type > Change to code cell type) and run it after everything is processed.

In [ ]:
plot_directory = os.path.join(os.getcwd(),'plot')
if not os.path.isdir(plot_directory): os.makedirs(plot_directory)
for i in tqdm(range(len(grasslands_to_inspect)), desc="Plotting newly detected hpr grasslands"):
    grassland = grasslands_to_inspect.iloc[i]
    
    if grassland.ditch_buffer_fraction > .65:          
        clipped_image = utils.clip_raster(VITO_map, grassland.geometry.geoms)
    
        fig, ax = plt.subplots()
        # Display the raster image
        rasterio.plot.show(clipped_image, ax=ax, cmap='gray_r')
        # Plot the detected lines
        grasslands_to_inspect[i:i+1].plot(ax=ax, edgecolor='green', lw=1, label='Perceel', facecolor='none')
        buffer_zones.iloc[i:i+1].plot(ax=ax, color='orange', alpha=0.5, label='30m Buffer')
        ditches.iloc[i:i+1].plot(ax=ax, color='purple', linewidth=2, label='Grachtjes')
        
        fig.suptitle("Detected Lines on Georeferenced Image")
        ax.set_title(f"Ditches detected with a buffer zone fraction of {grassland.ditch_buffer_fraction}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
    
        fig.savefig(f'plot/KB20_grassland-{grassland.OBJECTID}.png')
        plt.close(fig)